In [ ]:

import os
import pandas as pd
import xlrd  # dong nay de doc file xlsm
import glob
pd.set_option('display.max_columns',500,'float_format', '{:,.2f}'.format) #hien het cac dong, format dau phay, 2 so 0
pd.set_option('display.max_rows', 500)
path_yearly=     '../data/raw/Bank/yearly/'   # use your path
path_quarterly = '../data/raw/Bank/Q3/'  # use your path
data_clean_bank_yearly = '../data/clean/bank/yearly/'
data_clean_bank_quarterly = '../data/clean/bank/quarterly/'
# thay đổi path ở dòng path và data_clean để lấy dữ liệu yearly hoặc quarterly
path = path_quarterly
data_clean=   data_clean_bank_quarterly
report_files = glob.glob(path + "*.xlsm")

list_BS = []
list_IS = []
list_CF = []
list_FS = []
list_notes = []
list_growth = []
list_tindung_growth_CP = []

for filename in report_files:
    df = pd.read_excel(filename, sheet_name=None, header=None)
    df_BS = df['Cân đối kế toán'] # lay sheet CDKT
    Ma_CP = df_BS.loc[7,1]   # lay ma cp o dong 8, cot B
    df_BS = df['Cân đối kế toán'] # lay sheet CDKT
    Ma_CP = df_BS.loc[7,1]   # lay ma cp o dong 8, cot B
    Ma_CP
    #df_BS.dropna(how='all', inplace = True)
    row_tieu_de = int(df_BS[df_BS[0] == 'Tiêu đề'].index[0])# tim row Tiêu đề
    begin_BS = row_tieu_de + 6
    row_CTCP_Fin_row = int(df_BS[df_BS[0] == 'CÔNG TY CỔ PHẦN FIINGROUP'].index[0])# tim row CÔNG TY CỔ PHẦN FIINGROUP
    end_BS = row_CTCP_Fin_row - 1
    BS = df_BS.iloc[begin_BS:end_BS]
    BS.iloc[0,0]= "items"
    BS.iloc[1,0]= "Balance_sheet"
    BS.columns = BS.iloc[0] # lay dong dau tien lam ten column
    BS.drop(BS.index[0],inplace= True) # bo dong dau tien
    BS.set_index('items', inplace = True) # set index cột items
    #BS_drop = BS.drop('2022F',axis=1) # loại cột Forecást 2022
    #BS.loc['Tín dụng'] = BS.loc['Cho vay khách hàng'] + BS.loc['Tiền gửi tại các TCTD khác và cho vay các TCTD khác']
    BS.loc['Tín dụng growth'] = BS.loc['Cho vay khách hàng'].pct_change(axis = 'rows',periods=3)*100  # tin dung 2 quy

    idx = 0
    # the

    BS_T = BS.T
    BS_T.insert(loc = idx, column= 'CP', value= Ma_CP) # them cot CP o cot so 1
    list_BS.append(BS_T)
    BS_pct= BS.loc['TỔNG TÀI SẢN':].pct_change(axis = 'columns')*100
    #BS_pct
    BS_growth = BS_pct.loc[['TỔNG TÀI SẢN','     Cho vay khách hàng','     Tiền gửi của khách hàng','Vốn chủ sở hữu','Tín dụng growth']]

    #%% INCOME STATEMENT
    df_IS = df['Kết quả Kinh doanh'] # Kết quả Kinh doanh
    row_tieu_de = int(df_IS[df_IS[0] == 'Tiêu đề'].index[0])# tim row Tiêu đề
    begin_IS = row_tieu_de + 6
    row_CTCP_Fin_row = int(df_IS[df_IS[0] == 'CÔNG TY CỔ PHẦN FIINGROUP'].index[0])# tim row CÔNG TY CỔ PHẦN FIINGROUP
    end_IS = row_CTCP_Fin_row - 1
    IS = df_IS.iloc[begin_IS:end_IS]
    IS.iloc[0,0]= "items"
    IS.iloc[1,0] = "Income_statement"
    IS.columns = IS.iloc[0] # lay dong dau tien lam ten column
    IS.drop(IS.index[0],inplace= True) # bo dong dau tien
    #FS_T.insert(loc = idx, column= 'CP', value= Ma_CP) # them cot CP o cot so 1
    IS.set_index('items', inplace = True) #
    #IS_drop = IS.drop('2022F',axis=1) # loại cột Forecást 2022
    IS_T = IS.T
    IS_T.insert(loc = idx, column= 'CP', value= Ma_CP) # them cot CP o cot so 1
    list_IS.append(IS_T)
    IS_pct= IS.loc['Thu nhập lãi và các khoản thu nhập tương tự':,'Q2/2020':].pct_change(axis = 'columns')*100
    #IS_pct
    IS_growth = IS_pct.loc[['Thu nhập lãi thuần','     Thu nhập từ hoạt động dịch vụ','Chi phí hoạt động','Tổng thu nhập hoạt động','LN thuần từ hoạt động kinh doanh trước CF dự phòng rủi ro tín dụng','Tổng lợi nhuận trước thuế','Lợi nhuận sau thuế','Lãi cơ bản trên cổ phiếu (VND)']]
    #IS_growth
    ratio_growth = pd.concat([BS_growth, IS_growth], axis=0)
    ratio_growth.rename_axis("ratio growth", inplace=True)
    growth =  ratio_growth.T
    #growth
    idx = 0
    growth.insert(loc = idx, column= 'CP', value= Ma_CP)
    #growth
    list_growth.append(growth)
    #CASHFLOW DIRECT
    df_CF = df['Lưu chuyển tiền tệ'] # Lưu chuyển tiền tệ
    row_tieu_de = int(df_CF[df_CF[0] == 'Tiêu đề'].index[0]) #tim tieu de
    begin_CF = row_tieu_de + 6
    #row_gian_tiep = int(df_CF[df_CF[0] == 'Gián Tiếp '].index[0])# tim
    #row_CTCP_Fin_row = int(df_CF[df_CF[0] == 'CÔNG TY CỔ PHẦN FIINGROUP'].index[0])# tim row CÔNG TY CỔ PHẦN FIINGROUP
    #end_CF = row_CTCP_Fin_row - 1
    row_end_CF_dir = int(df_CF[df_CF[0] == 'Tiền và các khoản tương đương tiền tại thời điểm cuối kỳ'].index[0]) # tim row cuối CF Direct
    CF_Dir = df_CF.iloc[begin_CF:row_end_CF_dir + 1] # Cashflow statement truc tiep
    CF_Dir.iloc[0,0]= "items"
    CF_Dir.iloc[1,0]= "CF trực tiếp"
    CF_Dir.columns = CF_Dir.iloc[0] # lay dong dau tien lam ten column
    CF_Dir.drop(CF_Dir.index[0],inplace= True) # bo dong dau tien
    CF_Dir.set_index('items', inplace = True) #
    CF_T = CF_Dir.T
    CF_T.insert(loc = idx, column= 'CP', value= Ma_CP) # them cot CP o cot so 1
    list_CF.append(CF_T)
    # CASHFLOW INDIRECT
    #CF_inDir = df_CF.iloc[row_gian_tiep:end_CF] # Cashflow statement gian tiep
    #CF_inDir.iloc[0,0]= "items"
    #CF_inDir.iloc[1,0]= "CF gián tiếp"
    #CF_inDir.columns = CF_inDir.iloc[0] # lay dong dau tien lam ten column
    #CF_inDir.drop(CF_inDir.index[0],inplace= True) # bo dong dau tien
    #CF_inDir.set_index('items', inplace = True) #
    #CF_inDir
    # notes
    df_note = df['Thuyết minh'] # Thuyết minh
    row_tieu_de = int(df_note[df_note[0] == 'Tiêu đề'].index[0]) #tim tieu de
    begin_note = row_tieu_de + 6
    row_CTCP_Fin_row = int(df_note[df_note[0] == 'CÔNG TY CỔ PHẦN FIINGROUP'].index[0])# tim row CÔNG TY CỔ PHẦN FIINGROUP
    end_note = row_CTCP_Fin_row - 1
    notes = df_note.iloc[begin_note:end_note] # C
    notes.iloc[1,0]= "Notes"
    notes.iloc[0,0]= "items"
    notes.iloc[1,0]= "notes"
    notes.columns = notes.iloc[0] # lay dong dau tien lam ten column
    notes.drop(notes.index[0],inplace= True) # bo dong dau tien
    notes.set_index('items', inplace = True) #
    notes_T = notes.T
    notes_T.insert(loc = idx, column= 'CP', value= Ma_CP) # them
    list_notes.append(notes_T)
    FS = pd.concat([BS, IS, CF_Dir,notes], axis=0)
    #FS.loc['CP'] = Ma_CP # them dong co phieu
    #FS.index = FS.index + 1
    #FS.sort_index(inplace = True)
    FS.to_excel(data_clean + Ma_CP + '_clean.xlsx')
    FS.index = FS.index.str.strip() # cat cac space o dau va cuoi str
    FS
    FS_T = FS.T # BS.transpose() trandoi col to row
    #BS_T.columns = BS_T.iloc[0] # lay dong dau tien lam ten column
    #BS_T.drop(BS_T.index[0],inplace= True) # bo dong dau tien
    #idx = 0
    FS_T.insert(loc = idx, column= 'CP', value= Ma_CP) # them cot CP o cot so 1
    FS_T.to_excel(data_clean + Ma_CP + '_clean_transpose.xlsx')
    list_FS.append(FS_T)
    #FS_all = pd.concat([FS_all,FS_T], axis = 0, ignore_index = True)
FS_all = pd.concat(list_FS, axis = 0 )
BS_all = pd.concat(list_BS, axis = 0 )
IS_all = pd.concat(list_IS, axis = 0 )
CF_all = pd.concat(list_CF, axis = 0 )
notes_all = pd.concat(list_notes, axis = 0 )
FS_all.rename_axis("BCTC_nganh", inplace = True)
BS_all.rename_axis("BS_nganh", inplace = True)
IS_all.rename_axis("IS_nganh", inplace = True)
CF_all.rename_axis("CF_nganh", inplace = True)
FS_all.to_excel(data_clean + 'FS_all' + '_clean_transpose.xlsx')

growth_rate = pd.concat(list_growth, axis = 0 )
growth_rate

#FS_all


In [ ]:
report_ticker_year